In [1]:
import pandas as pd
import markovify 
import spacy
import re

import warnings
warnings.filterwarnings('ignore')

from time import time
import gc

In [2]:
curr_dir = r'D:\Python\MachineLearning\nyt-comments'
df1 = pd.read_csv(curr_dir + '\CommentsJan2017.csv')
df2 = pd.read_csv(curr_dir + '\CommentsFeb2017.csv')
df3 = pd.read_csv(curr_dir + '\CommentsMarch2017.csv')
df4 = pd.read_csv(curr_dir + '\CommentsApril2017.csv')
df5 = pd.read_csv(curr_dir + '\CommentsMay2017.csv')
df6 = pd.read_csv(curr_dir + '\CommentsJan2018.csv')
df7 = pd.read_csv(curr_dir + '\CommentsFeb2018.csv')
df8 = pd.read_csv(curr_dir + '\CommentsMarch2018.csv')
df9 = pd.read_csv(curr_dir + '\CommentsApril2018.csv')
comments = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
comments.drop_duplicates(subset='commentID', inplace=True)
comments.head(3)

,approveDate,articleID,articleWordCount,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,...,status,timespeople,trusted,typeOfMaterial,updateDate,userDisplayName,userID,userLocation,userTitle,userURL
0,1483455908,58691a5795d0e039260788b9,1324.0,For all you Americans out there --- still rejo...,20969730.0,20969730.0,<br/>,comment,1.483426e+09,1.0,...,approved,1.0,0.0,News,1483455908,N. Smith,64679318.0,New York City,NaN,NaN
1,1483455656,58691a5795d0e039260788b9,1324.0,Obamas policies may prove to be the least of t...,20969325.0,20969325.0,<br/>,comment,1.483417e+09,1.0,...,approved,1.0,0.0,News,1483455656,Kilocharlie,69254188.0,Phoenix,NaN,NaN
2,1483455655,58691a5795d0e039260788b9,1324.0,Democrats are comprised of malcontents who gen...,20969855.0,20969855.0,<br/>,comment,1.483431e+09,1.0,...,approved,1.0,0.0,News,1483455655,Frank Fryer,76788711.0,Florida,NaN,NaN


In [3]:
comments.shape

(2118617, 34)

In [4]:
comments.sectionName.value_counts()[:5]


Unknown          1096761
Politics          479701
Sunday Review     143849
Europe             46844
Middle East        32385
Name: sectionName, dtype: int64

In [5]:
def preprocess(comments):
    commentBody = comments.loc[comments.sectionName=='Politics', 'commentBody']
    commentBody = commentBody.str.replace("(<br/>)", "")
    commentBody = commentBody.str.replace('(<a).*(>).*(</a>)', '')
    commentBody = commentBody.str.replace('(&amp)', '')
    commentBody = commentBody.str.replace('(&gt)', '')
    commentBody = commentBody.str.replace('(&lt)', '')
    commentBody = commentBody.str.replace('(\xa0)', ' ')  
    return commentBody

In [6]:
commentBody = preprocess(comments)
commentBody.shape

(479701,)

In [7]:
del comments, df1, df2, df3, df4, df5, df6, df7, df8
gc.collect()

366

In [8]:
commentBody.sample().values[0]
#A random comment from the dataset:

'No, the best thing that the Democrats could do is to call for a roll call vote, so that every Representative would have to be shown on the record as having voted for or against this bill. This would make creating those 2018 election ads very easy indeed.'

In [10]:
start_time = time()
comments_generator = markovify.Text(commentBody, state_size = 5)
print("Run time for training the generator : {} seconds".format(round(time()-start_time, 2)))

KeyboardInterrupt: 

In [ ]:
def generate_comments(generator, number=10, short=False):
    count = 0
    while count < number:
        if short:
            comment = generator.make_short_sentence(140)
        else:
            comment = generator.make_sentence()
        if comment:
            count += 1
            print("Comment {}".format(count))
            print(comment)
            print()

In [ ]:
generate_comments(comments_generator)

In [ ]:
nlp = spacy.load('en_core_web_sm')

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

In [ ]:
commentBody = preprocess(df9)
commentBody.shape

In [ ]:
del comments_generator, df9
gc.collect()

In [ ]:
#Getting More Comments
start_time = time()
comments_generator_POSified = POSifiedText(commentBody, state_size = 2)
print("Run time for training the generator : {} seconds".format(round(time()-start_time, 2)))

In [ ]:
generate_comments(comments_generator_POSified)